In [ ]:
#Installing and checking requiered packages

# !pip show rdflib pandas numpy scikit-learn shap lime matplotlib ipykernel
# !pip install rdflib shap lime


In [2]:
#imports
import gzip
from rdflib import Graph
import pandas as pd
import os
from tqdm import tqdm

In [ ]:

g = Graph()
for i in range(10):  
    file = f"X-AI_MP-/data/am_stripped0{i}.nt"
    g.parse(file, format="nt")


In [1]:
for s, g, p in g:
    print(f"S: {s}, G: {g}, P: {p}")

NameError: name 'g' is not defined

In [ ]:


import pandas as pd


train_entities = pd.read_csv('X-AI_MP-/data/trainingSet.tsv', sep='\t', names=['entity', 'label'], skiprows=1)
test_entities = pd.read_csv('X-AI_MP-/data/testSet.tsv', sep='\t', names=['entity', 'label'], skiprows=1)

# Convert numeric entity IDs into RDF-style URIs
def id_to_uri(x):
    x = int(float(x))  
    return f"http://purl.org/collections/nl/am/e-{x}"

train_entities['entity'] = train_entities['entity'].apply(id_to_uri)
test_entities['entity'] = test_entities['entity'].apply(id_to_uri)
all_entities = pd.concat([train_entities, test_entities], ignore_index=True)

# Check sample
print("✅ Sample converted entity URIs:")
print(train_entities.head())


✅ Sample converted entity URIs:
                                                                              entity  \
http://purl.org/collections/nl/am/proxy-69550  http://purl.org/collections/nl/am/e-1   
http://purl.org/collections/nl/am/proxy-6266   http://purl.org/collections/nl/am/e-2   
http://purl.org/collections/nl/am/proxy-47358  http://purl.org/collections/nl/am/e-3   
http://purl.org/collections/nl/am/proxy-24708  http://purl.org/collections/nl/am/e-5   
http://purl.org/collections/nl/am/proxy-54032  http://purl.org/collections/nl/am/e-7   

                                                                                   label  
http://purl.org/collections/nl/am/proxy-69550  http://purl.org/collections/nl/am/t-22503  
http://purl.org/collections/nl/am/proxy-6266   http://purl.org/collections/nl/am/t-15459  
http://purl.org/collections/nl/am/proxy-47358  http://purl.org/collections/nl/am/t-22503  
http://purl.org/collections/nl/am/proxy-24708  http://purl.org/collections/

In [38]:
print(all_entities)

                                      entity  \
0      http://purl.org/collections/nl/am/e-1   
1      http://purl.org/collections/nl/am/e-2   
2      http://purl.org/collections/nl/am/e-3   
3      http://purl.org/collections/nl/am/e-5   
4      http://purl.org/collections/nl/am/e-7   
..                                       ...   
995  http://purl.org/collections/nl/am/e-989   
996  http://purl.org/collections/nl/am/e-991   
997  http://purl.org/collections/nl/am/e-992   
998  http://purl.org/collections/nl/am/e-995   
999  http://purl.org/collections/nl/am/e-998   

                                         label  
0    http://purl.org/collections/nl/am/t-22503  
1    http://purl.org/collections/nl/am/t-15459  
2    http://purl.org/collections/nl/am/t-22503  
3    http://purl.org/collections/nl/am/t-15579  
4    http://purl.org/collections/nl/am/t-22503  
..                                         ...  
995  http://purl.org/collections/nl/am/t-22503  
996  http://purl.org/collection

In [64]:
# from collections import defaultdict
# import pandas as pd

# # Create a dict of dicts: entity ➜ predicate ➜ count
# entity_features = defaultdict(lambda: defaultdict(int))

# for s, p, o in g:
#     subject = str(s)
#     predicate = str(p)
#     if subject in all_entities['entity'].values:
#         entity_features[subject][predicate] += 1
        
from collections import defaultdict

target_uris = set(all_entities['entity'])

entity_features = defaultdict(lambda: defaultdict(int))

for s, p, o in g:
    o = str(o)
    if o in target_uris:
        p_short = str(p).split('#')[-1].split('/')[-1]  # Simplify predicate
        entity_features[o][p_short] = 1  # Use object as the target entity


In [66]:
# Convert to tabular format
# feature_df = pd.DataFrame.from_dict(entity_features, orient='index').fillna(0)
# feature_df.index.name = 'entity'
# feature_df.reset_index(inplace=True)

feature_df = pd.DataFrame.from_dict(entity_features, orient='index').fillna(0)
feature_df.index.name = 'entity'
feature_df.reset_index(inplace=True)

train_df = train_entities.merge(feature_df, on='entity', how='left').fillna(0)
test_df = test_entities.merge(feature_df, on='entity', how='left').fillna(0)



In [67]:
print("New feature_df shape:", feature_df.shape)


New feature_df shape: (0, 1)


In [53]:
train_df = train_entities.merge(feature_df, on='entity', how='left').fillna(0)
test_df = test_entities.merge(feature_df, on='entity', how='left').fillna(0)


In [54]:
train_df.head(3)

,entity,label
0,http://purl.org/collections/nl/am/e-1,http://purl.org/collections/nl/am/t-22503
1,http://purl.org/collections/nl/am/e-2,http://purl.org/collections/nl/am/t-15459
2,http://purl.org/collections/nl/am/e-3,http://purl.org/collections/nl/am/t-22503


In [55]:
X_train = train_df.drop(columns=['entity', 'label'])
y_train = train_df['label']

X_test = test_df.drop(columns=['entity', 'label'])
y_test = test_df['label']

In [60]:
print("feature_df shape:", feature_df.shape)
print("feature_df.head():\n", feature_df.head())


print("Sample feature_df entity:", feature_df['entity'].head().tolist())
print("Sample train_entities entity:", train_entities['entity'].head().tolist())


feature_df shape: (0, 1)
feature_df.head():
 Empty DataFrame
Columns: [entity]
Index: []
Sample feature_df entity: []
Sample train_entities entity: ['http://purl.org/collections/nl/am/e-1', 'http://purl.org/collections/nl/am/e-2', 'http://purl.org/collections/nl/am/e-3', 'http://purl.org/collections/nl/am/e-5', 'http://purl.org/collections/nl/am/e-7']


In [61]:
from itertools import islice

# Print 10 unique RDF subjects
print("🔍 Sample RDF subjects:")
for s in islice(set(str(triple[0]) for triple in g), 10):
    print(s)


🔍 Sample RDF subjects:
N77cf5480e1304c17b5c83ccea7a2708c
Nee0a2ad9078948bdb17d905dcb675951
Nb0b7263c17934dafa72b1debafb40247
N33f4013f15be4397aac3c36a0eedafb0
N352907f6f32d417a876bef5b0f0dea51
http://purl.org/collections/nl/am/p-32894
N1d0aa056769741cd8a208480746c2503
N58ccc0dc304f46a9ae5ef2f5adc80d28
N5b6b4f1d105d49c99a5e0434432d512a
N1a722da38e2340d0995d07e83791a91d


In [62]:
rdf_subjects = {str(s) for s, _, _ in g}
train_subjects = set(train_entities['entity'])

matched = rdf_subjects & train_subjects
print(f"✅ Matching entities in RDF: {len(matched)} / {len(train_subjects)}")


✅ Matching entities in RDF: 0 / 802


In [63]:
# Try partial matching to test if RDF URIs have extra characters
for s, p, o in g:
    s_str = str(s)
    if any(e in s_str for e in train_subjects):
        print("🔎 Possible fuzzy match found:", s_str)
        break


In [51]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)


In [52]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=1000, random_state=0)
model.fit(X_train, y_train_encoded)


ValueError: at least one array or dtype is required